In [1]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
import rasterio as ra
from PIL import Image

In [2]:
image_path='./py/KMeanMat_5E05.tif'

In [3]:
def raster_write(output_file,data,count,crs,transform):#(路径，矩阵数据，波段数，坐标系统，仿射关系）
    writer=ra.open(output_file,mode='w+',driver='GTiff',height=data.shape[-2],width=data.shape[-1],
                   count=count,dtype=np.float32,crs=crs,transform=transform)
    for i in range(1,count+1):
        writer.write(data[i-1],i)
    writer.close()

with ra.open(image_path) as originial_image:
    data=originial_image.read() #栅格数据
    transform=originial_image.transform #仿射关系
    crs=originial_image.crs #坐标系统
    count=originial_image.count #波段数


In [4]:
img=data[0] #读取灰度图

In [5]:
raster_write('./py/result.tif',data,1,crs,transform)

In [6]:
#检查读取是否成功
band,rows,cols=data.shape
print(data.shape)

(1, 25162, 25000)


In [7]:
#进行高斯平滑处理
blurred_image=cv2.GaussianBlur(img,(3,3),0)

In [8]:
#Sobel边界检测运算
Gx=cv2.Sobel(blurred_image,cv2.CV_64F,1,0,ksize=3)
Gy=cv2.Sobel(blurred_image,cv2.CV_64F,0,1,ksize=3)
#Gradient magnitude
G=np.sqrt(Gx**2+Gy**2)

In [9]:
#标准化至0-255范围
Gx=np.uint8(255*np.abs(Gx)/np.max(Gx))
Gy = np.uint8(255 * np.abs(Gy) / np.max(Gy))
G = np.uint8(255 * G / np.max(G))

In [10]:
image=Image.fromarray(G)
image.save('./py/edge_detected.tiff')


In [13]:
unique_values=np.unique(G)
print(f"唯一取值: {unique_values}")

唯一取值: [  0   1   2   3   4   5   6   7   8   9  10  11  12  13  14  15  16  17
  18  19  20  21  22  23  24  25  26  27  28  29  30  31  32  33  34  35
  36  37  38  39  40  41  42  43  44  45  46  47  48  49  50  51  52  53
  54  55  56  57  58  59  60  61  62  63  64  65  66  67  68  69  70  71
  72  73  74  75  76  77  78  79  80  81  82  83  84  85  86  87  88  89
  90  91  92  93  94  95  96  97  98  99 100 101 102 103 104 105 106 107
 108 109 110 111 112 113 114 115 116 117 118 119 120 121 122 123 124 125
 126 127 128 129 130 131 132 133 134 135 136 137 138 139 140 141 142 143
 144 145 146 147 148 149 150 151 152 153 154 155 156 157 158 159 160 161
 162 163 164 165 166 167 168 169 170 171 172 173 174 175 176 177 178 179
 180 181 182 183 184 185 186 187 188 189 190 191 192 193 194 195 196 197
 198 199 200 201 202 203 204 205 206 207 208 209 210 211 212 213 214 215
 216 217 218 219 220 221 222 223 224 225 226 227 228 229 230 231 232 233
 234 235 236 237 238 239 240 241 242 243 244 

In [15]:
standard_G=G
standard_G[standard_G!=0]=255

In [16]:
image=Image.fromarray(standard_G)
image.save('./py/edge_detected_standardlized.tiff')